In [33]:
import pandas as pd

file_path= r"C:\Users\USER\Desktop\사용할만한 csv파일\2008년 남자데이터 (결측치 대치,k=5).CSV"
X=pd.read_csv(file_path)

종속변수 (Y), 독립변수 (X) 분할

In [34]:
Y=X['fat']
X=X.drop(['fat', 'DW_WBT_pFT'], axis=1)    #얻을 수 없는 column제거
print("컬럼 개수 :", len(X.columns))

컬럼 개수 : 88


training set, test set 분할

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, shuffle=True, random_state=42)

전체특성으로 훈련시킨 모델

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score

# 모델 생성
model = DecisionTreeClassifier(random_state=42)

# 모델 학습
model.fit(X_train, Y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(Y_test, y_pred)
print("모델 1 테스트 세트 정확도 (모든 특성):", accuracy)



모델 1 테스트 세트 정확도 (모든 특성): 0.5270900609974883


filter approach로 feature selection

1. 교차검정

In [37]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

max_features = X_train.shape[1]  # 데이터의 전체 특성 개수
feature_range = range(1, max_features+1)

best_score = float('inf')  # 초기값을 무한대로 설정
best_num_features = 0
best_selected_features = None

for num_features in feature_range:
    # 모델 생성
    model = DecisionTreeRegressor(random_state=42)
    feature_names = X_train.columns.tolist()
    # 특정 개수의 특성 선택
    selected_features = feature_names[:num_features]  # 선택된 특성의 이름 또는 인덱스
    
    # 선택된 특성만으로 데이터 준비
    X_selected_train = X_train[selected_features]
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_selected_train, Y_train, cv=5, scoring='neg_mean_absolute_error')
    
    # 평균 성능 계산 (음수로 저장되어 있으므로 양수로 변환)
    mean_score = -scores.mean()
    
    # 최적의 특성 개수 및 성능 갱신
    if num_features == 1 or mean_score < best_score:
        best_score = mean_score
        best_num_features = num_features
        best_selected_features = selected_features

print("Best number of features:", best_num_features)
print("Best score (MAE):", best_score)
print("Selected features:", best_selected_features)


Best number of features: 42
Best score (MAE): 0.5751233962041035
Selected features: ['DK9_dg', 'DE1_dg', 'BS5_21', 'L_DN_FAM', 'sex', 'LK_LB_IT', 'BS9_2', 'EC_wht_5', 'occp', 'LQ_4EQL', 'BE5_1', 'DI2_dg', 'BO3_07', 'N_FAT', 'DC6_dg', 'BP8', 'BH1', 'BO3_01', 'LQ4_00', 'L_OUT_FQ', 'BO3_05', 'LQ_5EQL', 'DC1_dg', 'pa_walk', 'DE1_32', 'LK_EDU', 'EC_lgw_2', 'sm_presnt', 'BO3_09', 'BS3_1', 'BMI', 'BS8_2', 'genertn', 'BD1_11', 'ho_incm5', 'BP1', 'pa_mid', 'N_PROT', 'DJ4_3', 'HE_ht', 'BO3_12', 'allownc']


In [38]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 특성 선택
selected_features_order =  selected_features
# 선택된 특성으로 데이터 준비
X_selected = X_train.loc[:, selected_features_order]  # 수정된 코드
Y_selected = Y_train

# 의사결정 트리 모델 생성
model2 = DecisionTreeClassifier(random_state=42)

# 모델 학습
model2.fit(X_selected, Y_selected)

# 테스트 데이터에 대한 예측 수행
X_test_selected = X_test.loc[:, selected_features_order]  # 수정된 코드
Y_pred = model2.predict(X_test_selected)

# 정확도 평가
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5270900609974883


분산 기반 filter approach

In [39]:
from sklearn.feature_selection import VarianceThreshold

# 특성 선택 기준 설정
threshold = 0.1  # 분산 기준값

# 분산 기반의 특성 선택 수행
selector = VarianceThreshold(threshold=threshold)
X_selected = selector.fit_transform(X_train)

# 선택된 특성 개수 출력
num_selected_features = X_selected.shape[1]
print("Number of selected features:", num_selected_features)

# 선택된 특성 인덱스 가져오기
selected_feature_indices = selector.get_support(indices=True)

# 선택된 특성 이름 가져오기
selected_feature_names = X.columns[selected_feature_indices]

# 선택된 특성 출력
print("Selected features:", selected_feature_names.tolist())

Number of selected features: 84
Selected features: ['DE1_dg', 'BS5_21', 'L_DN_FAM', 'sex', 'LK_LB_IT', 'BS9_2', 'EC_wht_5', 'occp', 'LQ_4EQL', 'BE5_1', 'DI2_dg', 'BO3_07', 'N_FAT', 'BP8', 'BH1', 'BO3_01', 'LQ4_00', 'L_OUT_FQ', 'BO3_05', 'LQ_5EQL', 'DC1_dg', 'pa_walk', 'DE1_32', 'LK_EDU', 'EC_lgw_2', 'sm_presnt', 'BO3_09', 'BS3_1', 'BMI', 'BS8_2', 'genertn', 'BD1_11', 'ho_incm5', 'BP1', 'pa_mid', 'N_PROT', 'DJ4_3', 'HE_ht', 'BO3_12', 'allownc', 'DK8_dg', 'DC5_dg', 'BD2_1', 'DI3_dg', 'BO3_03', 'DI1_2', 'DI2_2', 'BO3_04', 'N_WATER', 'L_BR_FAM', 'DJ4_dg', 'DI4_dg', 'DI1_dg', 'HE_wc', 'LQ_3EQL', 'N_EN', 'BO2_1', 'DN1_dg', 'DE2_dg', 'BO1_3', 'N_DIET', 'LQ_1EQL', 'N_CHO', 'DL1_dg', 'HE_wt', 'EC1_1', 'DF2_dg', 'DM1_dg', 'L_LN_FAM', 'incm5', 'edu', 'DC4_dg', 'D_1_1', 'LF_SAFE', 'house', 'EC_occp', 'N_INTK', 'marri_1', 'age', 'DC3_dg', 'BO3_02', 'DK4_dg', 'BS5_22', 'BO1_1']


In [40]:
# 특성 선택
selected_features_order =  selected_feature_names

# 선택된 특성으로 데이터 준비
X_selected = X_train.loc[:, selected_features_order]  # 수정된 코드
Y_selected = Y_train

# 의사결정 트리 모델 생성
model2 = DecisionTreeClassifier(random_state=42)

# 모델 학습
model2.fit(X_selected, Y_selected)

# 테스트 데이터에 대한 예측 수행
X_test_selected = X_test.loc[:, selected_features_order]  # 수정된 코드
Y_pred = model2.predict(X_test_selected)

# 정확도 평가
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)



Accuracy: 0.5249372084678866


wrapper approach (forward)

sfs 알고리즘 사용

In [41]:
from sklearn.feature_selection import SequentialFeatureSelector

# 의사결정 트리 모델 생성
model = DecisionTreeClassifier(random_state=42)

# SequentialFeatureSelector 객체 생성
sfs = SequentialFeatureSelector(estimator=model, direction='forward')

# 특성 선택 수행
X_selected = sfs.fit_transform(X_train, Y_train)

# 의사결정 트리 모델 학습
model.fit(X_selected, Y_train)

# 테스트 데이터에 대한 예측 수행
X_test_selected = sfs.transform(X_test)
Y_pred = model.predict(X_test_selected)

# 정확도 평가
accuracy = accuracy_score(Y_test, Y_pred)

# 선택된 특성
selected_feature_names = [feature for feature, selected in zip(X.columns, sfs.get_support()) if selected]

# 선택된 특성 개수
num_selected_features = len(selected_feature_names)

# 결과 출력
print("Best accuracy:", accuracy)
print("Selected features:", selected_feature_names)
print("Number of selected features:", num_selected_features)


c:\Users\USER\anaconda3\lib\site-packages\sklearn\feature_selection\_sequential.py:211: FutureWarning: Leaving `n_features_to_select` to None is deprecated in 1.0 and will become 'auto' in 1.3. To keep the same behaviour as with None (i.e. select half of the features) and avoid this warning, you should manually set `n_features_to_select='auto'` and set tol=None when creating an instance.
  warnings.warn(


Best accuracy: 0.5385719411553642
Selected features: ['DK9_dg', 'DE1_dg', 'L_DN_FAM', 'sex', 'EC_wht_5', 'BO3_07', 'DC6_dg', 'L_OUT_FQ', 'BO3_05', 'DC1_dg', 'DE1_32', 'LK_EDU', 'EC_lgw_2', 'BO3_09', 'BMI', 'DJ4_3', 'BO3_12', 'allownc', 'DK8_dg', 'DC5_dg', 'DI3_dg', 'DI1_2', 'BO3_04', 'L_BR_FAM', 'DJ4_dg', 'DI4_dg', 'DI1_dg', 'HE_wc', 'LQ_3EQL', 'BO2_1', 'DN1_dg', 'DE2_dg', 'BO1_3', 'LQ_1EQL', 'DC2_dg', 'DL1_dg', 'HE_wt', 'EC1_1', 'DF2_dg', 'DC4_dg', 'DC3_dg', 'LQ_2EQL', 'BO3_02', 'DK4_dg']
Number of selected features: 44


wrapper approach (backward)

In [42]:
from sklearn.feature_selection import RFE
model = DecisionTreeClassifier(random_state=42)
# 특성 개수 자동 선택
rfe = RFE(estimator=model, step=1)
rfe.fit(X_train, Y_train)

# 훈련 데이터에 대한 예측 값
X_selected = rfe.transform(X_train)
model.fit(X_selected, Y_train)

# 테스트 데이터에 대한 예측 값
X_test_selected = rfe.transform(X_test)
Y_pred = model.predict(X_test_selected)

# 정확도 평가
accuracy = accuracy_score(Y_test, Y_pred)

# 선택된 특성
selected_feature_names = X.columns[rfe.support_]

# 결과 출력
print("Best accuracy:", accuracy)
print("Selected features:", selected_feature_names)
selected_feature_count = len(selected_feature_names)
print("Selected feature count:", selected_feature_count)

Best accuracy: 0.5242195909580194
Selected features: Index(['BS5_21', 'L_DN_FAM', 'LK_LB_IT', 'BS9_2', 'EC_wht_5', 'occp',
       'LQ_4EQL', 'BE5_1', 'N_FAT', 'BP8', 'L_OUT_FQ', 'pa_walk', 'EC_lgw_2',
       'BS3_1', 'BMI', 'BS8_2', 'genertn', 'BD1_11', 'ho_incm5', 'BP1',
       'N_PROT', 'HE_ht', 'BD2_1', 'BO3_03', 'DI1_2', 'N_WATER', 'L_BR_FAM',
       'HE_wc', 'N_EN', 'BO2_1', 'N_CHO', 'HE_wt', 'DF2_dg', 'DM1_dg',
       'L_LN_FAM', 'incm5', 'edu', 'D_1_1', 'LF_SAFE', 'house', 'EC_occp',
       'N_INTK', 'age', 'BO1_1'],
      dtype='object')
Selected feature count: 44
